In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

In [2]:
stats = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference per game stats 2012-19.csv')
stats = stats[stats['Year'] >= 2016]
print(stats.shape)
stats.head()

(2545, 50)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
2386,Quincy Acy,PF,25,SAC,59,29,14.8,2.0,3.6,0.556,0.3,0.8,0.388,1.7,2.8,0.606,0.600,0.8,1.2,0.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2,2016,14.7,0.629,0.229,0.318,8.1,15.1,11.6,4.4,1.6,2.2,10.0,13.1,1.8,0.7,2.5,0.137,-0.2,0.2,-0.1,0.4
2387,Jordan Adams,SG,21,MEM,2,0,7.5,1.0,3.0,0.333,0.0,0.5,0.000,1.0,2.5,0.400,0.333,1.5,2.5,0.600,0.0,1.0,1.0,1.5,1.5,0.0,1.0,1.0,3.5,2016,17.3,0.427,0.167,0.833,0.0,15.9,7.6,31.9,10.3,0.0,19.6,30.5,0.0,0.0,0.0,0.015,-2.5,9.4,6.9,0.0
2388,Steven Adams,C,22,OKC,80,80,25.2,3.3,5.3,0.613,0.0,0.0,NaN,3.3,5.3,0.613,0.613,1.4,2.5,0.582,2.7,3.9,6.7,0.8,0.5,1.1,1.1,2.8,8.0,2016,15.5,0.621,0.000,0.460,12.5,16.1,14.4,4.3,1.0,3.3,14.1,12.6,4.2,2.3,6.5,0.155,0.0,0.2,0.2,1.1
2389,Arron Afflalo,SG,30,NYK,71,57,33.4,5.0,11.3,0.443,1.3,3.4,0.382,3.7,7.9,0.469,0.500,1.5,1.8,0.840,0.3,3.4,3.7,2.0,0.4,0.1,1.2,2.0,12.8,2016,10.9,0.531,0.298,0.164,1.1,11.0,6.1,9.9,0.5,0.3,8.7,17.9,1.8,0.9,2.7,0.055,-1.3,-1.6,-2.9,-0.5
2390,Alexis Ajinça,C,27,NOP,59,17,14.6,2.5,5.3,0.476,0.0,0.0,0.000,2.5,5.3,0.478,0.476,0.9,1.1,0.839,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2016,13.8,0.514,0.003,0.197,9.3,25.9,17.3,5.8,1.1,3.4,13.6,20.4,0.2,0.9,1.0,0.058,-3.4,-0.6,-4.0,-0.4


# Data Cleaning 

Due to trades, some players have multiple entries. The 'Tm' features highlights which team the player has played for. TOT indicates the combined stats in one season while the other entries are stats for the respective teams they played in. We will consider the stats of the entire season. We will also be considering only players who have played at least 15 games.

In [3]:
#Replacing TOT with the last team played
for y in [2016, 2017, 2018, 2019]:
    dup = []
    for i, k in enumerate(stats[stats['Year'] == y]['Player']):
        dup += [k] if stats.loc[stats['Year'] == y, 'Tm'].reset_index(drop = True)[i] == 'TOT' else []

    for j in dup:
        index = list(stats.loc[(stats['Player'] == j) & (stats['Year'] == y), 'Tm'].index)
        stats.loc[index[0], 'Tm'] = stats.loc[index[-1], 'Tm']
        
stats.drop_duplicates(['Player', 'Year'], keep = 'first', inplace = True)

In [4]:
#Dropping players who played less than 15 games
stats.reset_index(drop = True, inplace = True)

In [5]:
stats['Player'] = stats['Player'].str.replace('.', '')
stats['Player'] = stats['Player'].str.replace('*', '')

In [6]:
stats.replace(dict.fromkeys(['PG', 'SG', 'PG-SG', 'SG-PF', 'SF-SG', 'SG-SF'], 'Guard'), inplace = True)
stats.replace(dict.fromkeys(['SF', 'PF', 'PF-SF'], 'Forward'), inplace = True)
stats.replace(dict.fromkeys(['C', 'PF-C', 'C-PF'], 'Center'), inplace = True)

dummy = pd.get_dummies(stats['Pos'])
stats = pd.concat([stats, dummy], axis = 1)
stats.drop('Pos', axis = 1, inplace = True)
print(stats.shape)
stats.head()

(2032, 52)


,Player,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Center,Forward,Guard
0,Quincy Acy,25,SAC,59,29,14.8,2.0,3.6,0.556,0.3,0.8,0.388,1.7,2.8,0.606,0.600,0.8,1.2,0.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2,2016,14.7,0.629,0.229,0.318,8.1,15.1,11.6,4.4,1.6,2.2,10.0,13.1,1.8,0.7,2.5,0.137,-0.2,0.2,-0.1,0.4,0,1,0
1,Jordan Adams,21,MEM,2,0,7.5,1.0,3.0,0.333,0.0,0.5,0.000,1.0,2.5,0.400,0.333,1.5,2.5,0.600,0.0,1.0,1.0,1.5,1.5,0.0,1.0,1.0,3.5,2016,17.3,0.427,0.167,0.833,0.0,15.9,7.6,31.9,10.3,0.0,19.6,30.5,0.0,0.0,0.0,0.015,-2.5,9.4,6.9,0.0,0,0,1
2,Steven Adams,22,OKC,80,80,25.2,3.3,5.3,0.613,0.0,0.0,NaN,3.3,5.3,0.613,0.613,1.4,2.5,0.582,2.7,3.9,6.7,0.8,0.5,1.1,1.1,2.8,8.0,2016,15.5,0.621,0.000,0.460,12.5,16.1,14.4,4.3,1.0,3.3,14.1,12.6,4.2,2.3,6.5,0.155,0.0,0.2,0.2,1.1,1,0,0
3,Arron Afflalo,30,NYK,71,57,33.4,5.0,11.3,0.443,1.3,3.4,0.382,3.7,7.9,0.469,0.500,1.5,1.8,0.840,0.3,3.4,3.7,2.0,0.4,0.1,1.2,2.0,12.8,2016,10.9,0.531,0.298,0.164,1.1,11.0,6.1,9.9,0.5,0.3,8.7,17.9,1.8,0.9,2.7,0.055,-1.3,-1.6,-2.9,-0.5,0,0,1
4,Alexis Ajinça,27,NOP,59,17,14.6,2.5,5.3,0.476,0.0,0.0,0.000,2.5,5.3,0.478,0.476,0.9,1.1,0.839,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2016,13.8,0.514,0.003,0.197,9.3,25.9,17.3,5.8,1.1,3.4,13.6,20.4,0.2,0.9,1.0,0.058,-3.4,-0.6,-4.0,-0.4,1,0,0


# Missing Data:

In [7]:
stats.isnull().sum()[stats.isnull().sum() > 0]

FG%       9
3P%     164
2P%      28
eFG%      9
FT%      95
TS%       9
3PAr      9
FTr       9
TOV%      9
dtype: int64

In [8]:
stats[['3P', '3PA', '3P%']][stats['3P%'].isnull()].describe()

,3P,3PA,3P%
count,164.0,164.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [9]:
stats[['2P', '2PA', '2P%']][stats['2P%'].isnull()].describe()

,2P,2PA,2P%
count,28.0,28.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [10]:
stats[['FT', 'FTA', 'FT%']][stats['FT%'].isnull()].describe()

,FT,FTA,FT%
count,95.0,95.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


NAN because is all 0

# NBA All Stars

In [11]:
allstar = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All-Stars.csv')
allstar.head()

,Player,Year
0,Isiah Thomas,1986
1,Larry Bird,1986
2,Moses Malone,1986
3,Sidney Moncrief,1986
4,Julius Erving,1986


In [12]:
allstar['All Star'] = 1
allstar['Total All Star'] = 1

In [13]:
stats = stats.merge(allstar, on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i, 'Total All Star'] = stats.loc[stats['Player'] == i, 'Total All Star'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [14]:
print(stats.shape)
stats.head()

(2032, 54)


,Player,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Center,Forward,Guard,All Star,Total All Star
0,Quincy Acy,25.0,SAC,59.0,29.0,14.8,2.0,3.6,0.556,0.3,0.8,0.388,1.7,2.8,0.606,0.600,0.8,1.2,0.735,1.1,2.1,3.2,0.5,0.5,0.4,0.5,1.7,5.2,2016,14.7,0.629,0.229,0.318,8.1,15.1,11.6,4.4,1.6,2.2,10.0,13.1,1.8,0.7,2.5,0.137,-0.2,0.2,-0.1,0.4,0.0,1.0,0.0,0.0,0.0
304,Elijah Millsap,28.0,UTA,20.0,0.0,8.7,0.6,2.0,0.282,0.1,0.6,0.083,0.5,1.4,0.370,0.295,0.7,0.9,0.722,0.3,1.5,1.8,1.0,0.4,0.2,0.5,1.7,1.8,2016,6.5,0.384,0.308,0.462,4.1,19.5,11.8,16.2,2.1,1.5,16.1,14.9,-0.2,0.3,0.1,0.033,-5.9,2.1,-3.8,-0.1,0.0,0.0,1.0,0.0,0.0
255,Shane Larkin,23.0,BRK,78.0,17.0,22.4,2.8,6.4,0.442,0.5,1.4,0.361,2.3,5.1,0.464,0.481,1.1,1.4,0.776,0.4,1.9,2.3,4.4,1.2,0.2,1.9,1.6,7.3,2016,13.3,0.515,0.215,0.213,1.8,9.9,5.8,29.3,2.7,0.6,21.0,17.7,0.5,1.0,1.4,0.039,-1.6,-0.4,-2.0,0.0,0.0,0.0,1.0,0.0,0.0
256,Joffrey Lauvergne,24.0,DEN,59.0,15.0,17.6,3.3,6.4,0.513,0.2,0.9,0.245,3.1,5.5,0.557,0.530,1.1,1.2,0.899,1.3,3.6,4.9,0.9,0.2,0.3,0.9,1.9,7.9,2016,15.7,0.567,0.139,0.182,8.1,23.1,15.4,9.1,0.6,1.2,11.6,19.5,1.6,0.7,2.3,0.105,-0.3,-1.3,-1.6,0.1,1.0,0.0,0.0,0.0,0.0
257,Zach LaVine,20.0,MIN,82.0,33.0,28.0,5.3,11.7,0.452,1.5,3.9,0.389,3.8,7.8,0.482,0.516,2.0,2.5,0.793,0.3,2.5,2.8,3.1,0.8,0.2,1.9,2.4,14.0,2016,14.3,0.548,0.330,0.212,1.4,10.1,5.8,18.6,1.5,0.6,12.9,23.5,1.9,0.7,2.6,0.054,0.2,-1.6,-1.5,0.3,0.0,0.0,1.0,0.0,0.0


# NBA Awards

In [15]:
awards = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference Awards.csv')
awards.head()

,Unnamed: 0,Year,Player,MVP,Rookie of Year,DPOY,6th man of Year,Most Improved,Finals MVP
0,141,1953,Monk Meineke,NaN,1.0,NaN,NaN,NaN,NaN
1,140,1954,Ray Felix,NaN,1.0,NaN,NaN,NaN,NaN
2,139,1955,Bob Pettit,NaN,1.0,NaN,NaN,NaN,NaN
3,63,1956,Bob Pettit,1.0,NaN,NaN,NaN,NaN,NaN
4,138,1956,Maurice Stokes,NaN,1.0,NaN,NaN,NaN,NaN


In [16]:
awards.fillna(0, inplace = True)

In [17]:
awards['Total MVP'] = awards['MVP']
awards['Total Rookie of Year'] = awards['Rookie of Year']
awards['Total DPOY'] = awards['DPOY']
awards['Total 6th man of Year'] = awards['6th man of Year']
awards['Total Most Improved'] = awards['Most Improved']
awards['Total Finals MVP'] = awards['Finals MVP'] 

In [18]:
stats = stats.merge(awards.drop('Unnamed: 0', axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total MVP'] = stats.loc[stats['Player'] == i, 'Total MVP'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Rookie of Year'] = stats.loc[stats['Player'] == i, 'Total Rookie of Year'].cumsum()
    stats.loc[stats['Player'] == i , 'Total DPOY'] = stats.loc[stats['Player'] == i, 'Total DPOY'].cumsum()
    stats.loc[stats['Player'] == i , 'Total 6th man of Year'] = stats.loc[stats['Player'] == i, 'Total 6th man of Year'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Most Improved'] = stats.loc[stats['Player'] == i, 'Total Most Improved'].cumsum()
    stats.loc[stats['Player'] == i , 'Total Finals MVP'] = stats.loc[stats['Player'] == i, 'Total Finals MVP'].cumsum()
stats = stats[stats['Year'] >= 2016]

# All NBA Teams

In [19]:
allnba = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Teams.csv')
allnba.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Nikola Jokić
1,1,2019,1st,Giannis Antetokounmpo
2,2,2019,1st,Paul George
3,3,2019,1st,James Harden
4,4,2019,1st,Stephen Curry


In [20]:
allrookie = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Rookie Teams.csv')
allrookie.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Luka Dončić
1,1,2019,1st,Trae Young
2,2,2019,1st,Deandre Ayton
3,3,2019,1st,Jaren Jackson
4,4,2019,1st,Marvin Bagley


In [21]:
alldefensive = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference All NBA Defensive Teams.csv')
alldefensive.head()

,Unnamed: 0,Year,Team,Player
0,0,2019,1st,Rudy Gobert
1,1,2019,1st,Paul George
2,2,2019,1st,Giannis Antetokounmpo
3,3,2019,1st,Marcus Smart
4,4,2019,1st,Eric Bledsoe


In [22]:
allnba.loc[allnba['Team'] == '1st', 'All NBA 1st Team'] = 1
allnba.loc[allnba['Team'] == '2nd', 'All NBA 2nd Team'] = 1
try:
    allnba.loc[allnba['Team'] == '3rd', 'All NBA 3rd Team'] = 1
except:
    pass

In [23]:
allrookie.loc[allrookie['Team'] == '1st', 'All Rookie 1st Team'] = 1
allrookie.loc[allrookie['Team'] == '2nd', 'All Rookie 2nd Team'] = 1

In [24]:
alldefensive.loc[alldefensive['Team'] == '1st', 'All Defensive 1st Team'] = 1
alldefensive.loc[alldefensive['Team'] == '2nd', 'All Defensive 2nd Team'] = 1

In [25]:
allnba['Total All NBA 1st Team'] = allnba['All NBA 1st Team']
allnba['Total All NBA 2nd Team'] = allnba['All NBA 2nd Team']
try:
    allnba['Total All NBA 3rd Team'] = allnba['All NBA 3rd Team']
except:
    pass

allrookie['Total All Rookie 1st Team'] = allrookie['All Rookie 1st Team']
allrookie['Total All Rookie 2nd Team'] = allrookie['All Rookie 2nd Team']

alldefensive['Total All Defensive 1st Team'] = alldefensive['All Defensive 1st Team']
alldefensive['Total All Defensive 2nd Team'] = alldefensive['All Defensive 2nd Team']

In [26]:
allnba.fillna(0, inplace = True)
allrookie.fillna(0, inplace = True)
alldefensive.fillna(0, inplace = True)

In [27]:
stats = stats.merge(allnba.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All NBA 1st Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All NBA 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 2nd Team'].cumsum()
    try:
        stats.loc[stats['Player'] == i , 'Total All NBA 3rd Team'] = stats.loc[stats['Player'] == i, 'Total All NBA 3rd Team'].cumsum()
    except:
        pass
stats = stats[stats['Year'] >= 2016]

In [28]:
stats = stats.merge(allrookie.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All Rookie 1st Team'] = stats.loc[stats['Player'] == i, 'Total All Rookie 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All Rookie 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All Rookie 2nd Team'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [29]:
stats = stats.merge(alldefensive.drop(['Unnamed: 0', 'Team'], axis = 1), on = ['Player', 'Year'], how = 'outer').fillna(0).sort_values('Year')
for i in stats['Player'].unique():
    stats.loc[stats['Player'] == i , 'Total All Defensive 1st Team'] = stats.loc[stats['Player'] == i, 'Total All Defensive 1st Team'].cumsum()
    stats.loc[stats['Player'] == i , 'Total All Defensive 2nd Team'] = stats.loc[stats['Player'] == i, 'Total All Defensive 2nd Team'].cumsum()
stats = stats[stats['Year'] >= 2016]

In [30]:
stats.shape

(2035, 80)

# NBA Draft

In [31]:
draft = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference Draft.csv')
draft.head()

,Unnamed: 0,Pick,Player,College,Year
0,0,1.0,Larry Johnson,UNLV,1991
1,1,2.0,Kenny Anderson,Georgia Tech,1991
2,2,3.0,Billy Owens,Syracuse,1991
3,3,4.0,Dikembe Mutombo,Georgetown,1991
4,4,5.0,Steve Smith,Michigan State,1991


In [32]:
draft.rename({'Year' : 'Draft Year'}, axis = 1, inplace = True)

In [33]:
stats.shape

(2035, 80)

In [34]:
stats = stats.merge(draft[['Pick', 'Player', 'Draft Year']], how = 'left', on = 'Player')

In [35]:
stats.drop(stats[(stats['Player'] == 'Corey Brewer') & (stats['Draft Year'] == 1998)].index, inplace = True)
stats.drop(stats[(stats['Player'] == 'Marcus Thornton') & (stats['Draft Year'] == 2015)].index, inplace = True)
stats.drop(stats[(stats['Player'] == 'Justin Jackson') & (stats['Draft Year'] == 2018)].index, inplace = True)

In [36]:
stats.shape

(2035, 82)

In [37]:
stats['YOS'] = stats['Year'] - stats['Draft Year']

# Free Agency

In [38]:
free_agency = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference Free Agency.csv')
print(free_agency.shape)
free_agency.head()

(581, 5)


,Unnamed: 0,Player,New Team,Terms,Year
0,0,Kevin Durant,GSW,Signed 2-yr/$54M deal with Warriors,2016
1,1,LeBron James,CLE,Signed 3-yr/$100M deal with Cavaliers,2016
2,2,Hassan Whiteside,MIA,Signed 4-yr/$98M deal with Heat,2016
3,3,DeMar DeRozan,TOR,Signed 5-yr/$139M deal with Raptors,2016
4,4,Al Horford,BOS,Signed 4-yr/$113M deal with Celtics,2016


In [39]:
!pip install unidecode

     |████████████████████████████████| 238 kB 2.1 MB/s eta 0:00:01


In [40]:
import unidecode
for j in stats['Player'].unique():
    stats.replace(j, unidecode.unidecode(j), inplace = True)

for j in free_agency['Player'].unique():
    free_agency.replace(j, unidecode.unidecode(j), inplace = True)

In [41]:
free_agency['Player'] = free_agency['Player'].str.replace('.', '')

In [42]:
salary_2017_2018 = pd.read_csv('Web-Scrapping Scripts/Hoopshype.com NBA Salaries 2016-2018.csv')
salary_2017_2018.rename({'Actual_Salary' : 'Salary'}, axis = 1, inplace = True)
salary_2017_2018.drop('Adjusted_Salary', axis = 1, inplace = True)
salary_2017_2018.head()

,Unnamed: 0,Player,Salary,Year
0,0,LeBron James,30963450,2016
1,1,Al Horford,26540100,2016
2,2,Kevin Durant,26540100,2016
3,3,James Harden,26540100,2016
4,4,Mike Conley,26540100,2016


In [43]:
salary_2019 = pd.read_csv('Web-Scrapping Scripts/Hoopshype.com NBA Salaries 2019.csv')
salary_2019.head()

,Unnamed: 0,Player,Salary,Year
0,0,Stephen Curry,40231758,2019
1,1,Russell Westbrook,38506482,2019
2,2,Chris Paul,38506482,2019
3,3,James Harden,38199000,2019
4,4,John Wall,38199000,2019


In [44]:
salary = salary_2017_2018.append(salary_2019)
salary.head()

,Unnamed: 0,Player,Salary,Year
0,0,LeBron James,30963450,2016
1,1,Al Horford,26540100,2016
2,2,Kevin Durant,26540100,2016
3,3,James Harden,26540100,2016
4,4,Mike Conley,26540100,2016


In [45]:
inconsistent_in_stats = []
for i, k in enumerate(free_agency.merge(salary, how = 'left', on = 'Player')['Player']):
    inconsistent_in_stats += [k] if free_agency.merge(salary, how = 'left', on = 'Player')['Salary'].isnull()[i] else []
    
inconsistent_in_stats

['Maurice Harkless',
 'Nene Hilario',
 'Chase Budinger',
 'Rasual Butler',
 'Ish Smith',
 'PJ Hairston',
 'Robert Sacre',
 'Cory Jefferson',
 'Lou Amundson',
 'Chris Copeland',
 'Cleanthony Early',
 'Patty Mills',
 'Tim Hardaway',
 'Nene Hilario',
 'Glenn Robinson',
 'Gary Payton',
 'Ish Smith',
 'Nene Hilario',
 'JJ Barea',
 'Glenn Robinson',
 'Cameron Reynolds',
 'Timothe Luwawu-Cabarrot',
 'Ray Spalding',
 'Walt Lemon',
 'Amida Brimah',
 'Abdul Gaddy',
 'Kaiser Gates',
 'Terry Larrier',
 'Devyn Marble',
 'Elijah Millsap']

In [46]:
salary.replace({'Moe Harkless' : 'Maurice Harkless', 'Nenê' : 'Nene Hilario', 'Ishmael Smith' : 'Ish Smith', 'Patrick Mills' : 'Patty Mills',
                'Tim Hardaway Jr' : 'Tim Hardaway', 'Glenn Robinson III' : 'Glenn Robinson', 'Gary Payton II' : 'Gary Payton',
                'Jose Juan Barea' : 'JJ Barea', 'Timothe Luwawu' : 'Timothe Luwawu-Cabarrot'}, inplace = True)

In [47]:
free_agency1 = free_agency.merge(salary[['Player', 'Salary', 'Year']], how = 'inner', on = ['Player', 'Year'])
print(free_agency1.shape)
free_agency1.head()

(513, 6)


,Unnamed: 0,Player,New Team,Terms,Year,Salary
0,0,Kevin Durant,GSW,Signed 2-yr/$54M deal with Warriors,2016,26540100
1,1,LeBron James,CLE,Signed 3-yr/$100M deal with Cavaliers,2016,30963450
2,2,Hassan Whiteside,MIA,Signed 4-yr/$98M deal with Heat,2016,22116750
3,3,DeMar DeRozan,TOR,Signed 5-yr/$139M deal with Raptors,2016,26540100
4,4,Al Horford,BOS,Signed 4-yr/$113M deal with Celtics,2016,26540100


In [48]:
free_agency1.replace({'Nene Hilario' : 'Nene', 'Tim Hardaway' : 'Tim Hardaway Jr', 'Glenn Robinson' : 'Glenn Robinson III',
                      'Derrick Jones' : 'Derrick Jones Jr', 'Kelly Oubre' : 'Kelly Oubre Jr', 'Frank Mason' : 'Frank Mason III'},
                      inplace = True)

In [49]:
NBA_Data = stats.merge(free_agency1[['Player', 'New Team', 'Salary', 'Year']], how = 'inner')

In [50]:
print(NBA_Data.shape)
NBA_Data.head()

(499, 85)


,Player,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Center,Forward,Guard,All Star,Total All Star,MVP,Rookie of Year,DPOY,6th man of Year,Most Improved,Finals MVP,Total MVP,Total Rookie of Year,Total DPOY,Total 6th man of Year,Total Most Improved,Total Finals MVP,All NBA 1st Team,All NBA 2nd Team,All NBA 3rd Team,Total All NBA 1st Team,Total All NBA 2nd Team,Total All NBA 3rd Team,All Rookie 1st Team,All Rookie 2nd Team,Total All Rookie 1st Team,Total All Rookie 2nd Team,All Defensive 1st Team,All Defensive 2nd Team,Total All Defensive 1st Team,Total All Defensive 2nd Team,Pick,Draft Year,YOS,New Team,Salary
0,Ty Lawson,28.0,IND,66.0,13.0,21.4,2.1,5.4,0.393,0.5,1.6,0.333,1.6,3.7,0.419,0.444,0.9,1.3,0.686,0.4,1.4,1.8,3.6,0.8,0.1,1.4,1.3,5.7,2016,9.7,0.476,0.305,0.243,2.0,7.4,4.7,24.6,1.9,0.4,19.5,14.9,0.2,0.7,0.9,0.031,-2.4,-1.2,-3.6,-0.6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,2009.0,7.0,SAC,1315448
1,Christian Wood,20.0,PHI,17.0,0.0,8.5,1.3,3.1,0.415,0.2,0.6,0.364,1.1,2.5,0.429,0.453,0.8,1.2,0.619,0.8,1.5,2.2,0.2,0.3,0.4,0.1,0.6,3.6,2016,16.4,0.490,0.208,0.396,9.4,19.4,14.3,3.5,1.7,3.7,3.1,19.4,0.2,0.1,0.3,0.097,0.4,-0.6,-0.2,0.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,CHA,874636
2,Dion Waiters,24.0,OKC,78.0,15.0,27.6,3.6,9.1,0.399,1.1,3.1,0.358,2.5,6.0,0.421,0.460,1.4,2.0,0.713,0.5,2.1,2.6,2.0,1.0,0.2,1.5,1.8,9.8,2016,9.4,0.492,0.340,0.222,1.9,8.0,5.1,10.0,1.9,0.5,13.4,17.7,0.3,1.8,2.1,0.046,-2.4,-0.4,-2.8,-0.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2012.0,4.0,MIA,2898000
3,Dwyane Wade,34.0,MIA,74.0,73.0,30.5,7.3,16.0,0.456,0.1,0.6,0.159,7.2,15.4,0.468,0.459,4.4,5.5,0.793,1.1,3.0,4.1,4.6,1.1,0.6,2.7,1.6,19.0,2016,20.3,0.517,0.037,0.343,4.3,10.7,7.6,27.4,1.8,1.4,13.0,31.6,2.4,2.6,4.9,0.105,2.3,-0.4,1.9,2.2,0.0,0.0,1.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2003.0,13.0,CHI,23200000
4,Sasha Vujacic,31.0,NYK,61.0,25.0,14.9,1.7,4.5,0.383,0.9,2.3,0.364,0.9,2.2,0.403,0.477,0.5,0.6,0.821,0.4,2.0,2.4,1.4,0.6,0.1,0.7,1.4,4.9,2016,11.1,0.503,0.516,0.141,3.2,14.6,9.0,14.3,1.9,0.4,13.0,16.8,0.3,0.8,1.1,0.060,-1.5,0.3,-1.2,0.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,2004.0,12.0,NYK,1410598


In [51]:
NBA_Data['Year'].value_counts()

2019    148
2016    135
2018    109
2017    107
Name: Year, dtype: int64

In [52]:
NBA_Data.loc[NBA_Data['Year'] == 2016, 'Salary Cap'] = 94143000
NBA_Data.loc[NBA_Data['Year'] == 2017, 'Salary Cap'] = 99093000
NBA_Data.loc[NBA_Data['Year'] == 2018, 'Salary Cap'] = 101869000
NBA_Data.loc[NBA_Data['Year'] == 2019, 'Salary Cap'] = 109140000

In [53]:
NBA_Data['% of Cap'] = NBA_Data['Salary']/NBA_Data['Salary Cap']
NBA_Data['% of Cap'] = NBA_Data['% of Cap'] * 100
NBA_Data['% of Cap'] = NBA_Data['% of Cap'].round(1)

In [54]:
NBA_Data.to_csv('Free Agency Data 2016-2019.csv', index = False)